In [1]:
import pickle
from sympy import *
from binarize_features import *

language_model = 'roberta-base'
country_file = 'data/country_list_continents_new.csv'
occ_file = 'data/occupations_subset.csv'
binarizer = Binarizer(country_file, 5, occ_file)

### Loading

Load the extracted rule set and the background used from the corresponding files.

In [2]:
with open('data/' + language_model + '_rules_2dim.txt', 'rb') as f:
    h = pickle.load(f)
with open('data/background.txt', 'rb') as f:
    background = pickle.load(f)

In [3]:
all_rules = []
for rule in h:
    if rule not in background:
        expr = sstr(rule)
        all_rules.append(expr)
        print(expr)

Implies(v17 & v24, v22)
Implies(v23 & v24 & v7, v12)
Implies(v17 & v24, v8)
Implies(v17 & v24, v0)
Implies(v4, v25)
Implies(v17 & v24, v13)
Implies(v17 & v24, v16)
Implies(v0 & v10 & v22 & v24, v19)
Implies(v0 & v10 & v22 & v24, v15)
Implies(v0 & v10 & v22 & v24, v18)
Implies(v23 & v24 & v7, v14)
Implies(v0 & v10 & v22 & v24, v6)
Implies(v21 & v24, v18)
Implies(v21 & v24, v6)
Implies(v0 & v10 & v22 & v24, v25)
Implies(v0 & v10 & v22 & v24, v1)
Implies(v23 & v24 & v7, v21)
Implies(v23 & v24 & v7, v19)
Implies(v23 & v24 & v7, v15)
Implies(v23 & v24 & v7, v18)
Implies(v21 & v24, v17)
Implies(v21 & v24, v1)
Implies(v23 & v24 & v7, v6)
Implies(v0 & v10 & v22 & v24, v3)
Implies(v1, v25)
Implies(v23 & v24 & v7, v25)
Implies(v0 & v10 & v22 & v24, v12)
Implies(v21 & v24, v3)
Implies(v21 & v24, v12)
Implies(v23 & v24 & v7, v3)
Implies(v0 & v10 & v22 & v24, v14)
Implies(v21 & v24, v14)
Implies(v17 & v24, v18)
Implies(v0 & v10 & v22 & v24, v11)
Implies(v0 & v10 & v22 & v24, v21)
Implies(v23 & v24 

In [4]:
rules = []
for rule in all_rules:
    if '~' in rule:
        idx = []
        if not '(' in rule:
            idx.append(int(rule.replace('~v', '')))
        else:
            for s in rule.split(' '):
                if 'v' in s:
                    s = s.replace('~(', '')
                    s = s.replace(')', '')
                    idx.append(int(s.replace('v', '')))
        rules.append({'head' : '','body' : idx})
    else:
        body, head = rule.split(', ')
        body_idx = []
        for s in body.split(' '):
            if 'v' in s:
                s = s.replace('Implies(', '')
                body_idx.append(int(s.replace('v', '')))
        head_idx = head.replace('v', '').replace(')','')
        rules.append({'head' : head_idx, 'body' : body_idx})

In [5]:
total_lookup = []
for index in range(len(binarizer.age_containers) + 1):
    if index == 0:
        total_lookup.append("before " + str(binarizer.age_containers[index]))
    elif index == len(binarizer.age_containers):
        total_lookup.append("after " + str(binarizer.age_containers[index-1]))
    else:
        total_lookup.append("between " + str(binarizer.age_containers[index-1]) + " and " + str(binarizer.age_containers[index]))
for v in binarizer.continent_lookup.keys():
    total_lookup.append(v)
for v in binarizer.occupation_lookup.keys():
    total_lookup.append(v)
total_lookup.append('female')
total_lookup.append('male')
print({i : total_lookup[i] for i in range(len(total_lookup))})

{0: 'before 1892', 1: 'between 1892 and 1934', 2: 'between 1934 and 1956', 3: 'between 1956 and 1976', 4: 'after 1976', 5: 'North America', 6: 'Africa', 7: 'Europe', 8: 'Asia', 9: 'South America', 10: 'Oceania', 11: 'Eurasia', 12: 'Americas', 13: 'Australia', 14: 'fashion_designer', 15: 'nurse', 16: 'dancer', 17: 'priest', 18: 'footballer', 19: 'banker', 20: 'singer', 21: 'lawyer', 22: 'mathematician', 23: 'diplomat', 24: 'female', 25: 'male'}


In [6]:
string_rules = []
for rule in rules:
    s = ""
    if rule['head'] == '':
        s = s + "not "
    for variable in rule['body']:
        s = s + total_lookup[variable] + " & "
    s = s[:-2]
    if rule['head'] != '':
        s = s + " ---> " + total_lookup[int(rule['head'])]
    #if 'fashion_designer' not in s:
    string_rules.append(s)

In [7]:
string_rules.sort()
for s in string_rules:
    print(s)

Eurasia  ---> between 1892 and 1934
Eurasia  ---> male
Eurasia  ---> priest
South America  ---> male
after 1976  ---> male
before 1892 & Oceania & mathematician & female  ---> Africa
before 1892 & Oceania & mathematician & female  ---> Americas
before 1892 & Oceania & mathematician & female  ---> Asia
before 1892 & Oceania & mathematician & female  ---> Australia
before 1892 & Oceania & mathematician & female  ---> Eurasia
before 1892 & Oceania & mathematician & female  ---> Europe
before 1892 & Oceania & mathematician & female  ---> North America
before 1892 & Oceania & mathematician & female  ---> South America
before 1892 & Oceania & mathematician & female  ---> after 1976
before 1892 & Oceania & mathematician & female  ---> banker
before 1892 & Oceania & mathematician & female  ---> between 1892 and 1934
before 1892 & Oceania & mathematician & female  ---> between 1934 and 1956
before 1892 & Oceania & mathematician & female  ---> between 1956 and 1976
before 1892 & Oceania & mathem